In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk as nl
import tensorflow
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import regex as re
stop_word = set(stopwords.words('english'))
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-data/spam.csv


In [2]:
data=pd.read_csv('/kaggle/input/spam-data/spam.csv', encoding='latin')

In [3]:
data.head()
data1=data.iloc[:,:2]

In [12]:
def removing_punctuation(val):
    val2="".join([v for v in val if v not in string.punctuation])
    val1=val2.lower()
    return val2
def stop_words(val):
    val2=[v for v in val if v not in stop_word]
    return val2

In [5]:
data1['remove_punctuation']=data1['v2'].apply(lambda x:removing_punctuation(x))
data1.head()

/tmp/ipykernel_32/1104584760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['remove_punctuation']=data1['v2'].apply(lambda x:removing_punctuation(x))


,v1,v2,remove_punctuation
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...


In [6]:
data1['lower_case']=data1['remove_punctuation'].apply(lambda x:x.lower())
data1.head()

/tmp/ipykernel_32/201798011.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['lower_case']=data1['remove_punctuation'].apply(lambda x:x.lower())


,v1,v2,remove_punctuation,lower_case
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,free entry in 2 a wkly comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,nah i dont think he goes to usf he lives aroun...


In [9]:
data1['tokenization']=data1['lower_case'].apply(lambda x:word_tokenize(x))

0       [go, until, jurong, point, crazy, available, o...
1                          [ok, lar, joking, wif, u, oni]
2       [free, entry, in, 2, a, wkly, comp, to, win, f...
3       [u, dun, say, so, early, hor, u, c, already, t...
4       [nah, i, dont, think, he, goes, to, usf, he, l...
                              ...                        
5567    [this, is, the, 2nd, time, we, have, tried, 2,...
5568         [will, ì, b, going, to, esplanade, fr, home]
5569    [pity, was, in, mood, for, that, soany, other,...
5570    [the, guy, did, some, bitching, but, i, acted,...
5571                     [rofl, its, true, to, its, name]
Name: tokenization, Length: 5572, dtype: object

In [16]:
data1['remove_stop_words']=data1['tokenization'].apply(lambda x:stop_words(x))
data1['remove_stop_words']

0       [go, jurong, point, crazy, available, bugis, n...
1                          [ok, lar, joking, wif, u, oni]
2       [free, entry, 2, wkly, comp, win, fa, cup, fin...
3           [u, dun, say, early, hor, u, c, already, say]
4       [nah, dont, think, goes, usf, lives, around, t...
                              ...                        
5567    [2nd, time, tried, 2, contact, u, u, å£750, po...
5568                   [ì, b, going, esplanade, fr, home]
5569                     [pity, mood, soany, suggestions]
5570    [guy, bitching, acted, like, id, interested, b...
5571                                   [rofl, true, name]
Name: remove_stop_words, Length: 5572, dtype: object

In [20]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

In [21]:
def stemmer(val):
    val2=[porter_stemmer.stem(v) for v in val]
    return val2

In [22]:
data1['stemming']=data1['remove_stop_words'].apply(lambda x:stemmer(x))

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
countV = CountVectorizer()
transformed = countV.fit_transform(data['v2'].values)

In [31]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(transformed, data['v1'], test_size = 0.20)

In [33]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=300, n_jobs=2)
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=300, n_jobs=2)

In [35]:
y_pred = rf_classifier.predict(x_test)

In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       967
        spam       0.99      0.84      0.91       148

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115

